Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 3.130459
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.927408
Minibatch accuracy: 25.0%
Validation accuracy: 42.8%
Minibatch loss at step 100: 0.821461
Minibatch accuracy: 81.2%
Validation accuracy: 73.2%
Minibatch loss at step 150: 1.137578
Minibatch accuracy: 62.5%
Validation accuracy: 77.1%
Minibatch loss at step 200: 0.753682
Minibatch accuracy: 81.2%
Validation accuracy: 77.9%
Minibatch loss at step 250: 0.784285
Minibatch accuracy: 93.8%
Validation accuracy: 78.3%
Minibatch loss at step 300: 1.308770
Minibatch accuracy: 62.5%
Validation accuracy: 77.7%
Minibatch loss at step 350: 0.409027
Minibatch accuracy: 87.5%
Validation accuracy: 80.0%
Minibatch loss at step 400: 1.034429
Minibatch accuracy: 75.0%
Validation accuracy: 74.0%
Minibatch loss at step 450: 0.781351
Minibatch accuracy: 75.0%
Validation accuracy: 80.7%
Minibatch loss at step 500: 0.783812
Minibatch accuracy: 81.2%
Validation accuracy: 80.5%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [10]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  

  #helper functions to intialize weights, biases, convlution layer and max_pooling layer
  def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

  def bias_variable(val, shape):
    return tf.Variable(tf.constant(val, shape=shape))
                                   
  def conv2d(x , W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
  
  def max_pool(conv_layer):
    return tf.nn.max_pool(conv_layer, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')
                                   
  # Variables.
  weights_conv_1 = weight_variable([patch_size, patch_size, num_channels, depth])
  bias_conv1 = bias_variable(0.0,[depth])
  weights_conv_2 = weight_variable([patch_size, patch_size, depth, depth])
  biases_conv2 = bias_variable(1.0,[depth])
  weights_hidden_1 = weight_variable([image_size // 4 * image_size // 4 * depth, num_hidden])
  biases_hidden_1 = bias_variable(1.0,[num_hidden])
  weights_layer_final = weight_variable([num_hidden, num_labels])
  biases_final = bias_variable(1.0,[num_labels])
    
  
        
  def model(data):
    conv_1 = conv2d(data, weights_conv_1)
    hidden_conv_1 = tf.nn.relu(conv_1 + bias_conv1)
    max_pool_1 = max_pool(hidden_conv_1)
    conv_2 = conv2d(max_pool_1, weights_conv_2)
    hidden_conv_2 = tf.nn.relu(conv_2 + biases_conv2)
    max_pool_2 = max_pool(hidden_conv_2)
    shape = max_pool_2.get_shape().as_list()
    reshape = tf.reshape(max_pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden_3 = tf.nn.relu(tf.matmul(reshape, weights_hidden_1) + biases_hidden_1)
    return tf.matmul(hidden_3, weights_layer_final) + biases_final
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [11]:
num_steps = 2001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.894923
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.843723
Minibatch accuracy: 43.8%
Validation accuracy: 36.4%
Minibatch loss at step 100: 1.149565
Minibatch accuracy: 68.8%
Validation accuracy: 57.7%
Minibatch loss at step 150: 1.184085
Minibatch accuracy: 50.0%
Validation accuracy: 72.7%
Minibatch loss at step 200: 0.787223
Minibatch accuracy: 87.5%
Validation accuracy: 78.0%
Minibatch loss at step 250: 0.805691
Minibatch accuracy: 87.5%
Validation accuracy: 79.1%
Minibatch loss at step 300: 1.259999
Minibatch accuracy: 50.0%
Validation accuracy: 77.2%
Minibatch loss at step 350: 0.400711
Minibatch accuracy: 81.2%
Validation accuracy: 80.6%
Minibatch loss at step 400: 1.180632
Minibatch accuracy: 75.0%
Validation accuracy: 72.1%
Minibatch loss at step 450: 0.862439
Minibatch accuracy: 75.0%
Validation accuracy: 82.2%
Minibatch loss at step 500: 0.928611
Minibatch accuracy: 81.2%
Validation accuracy: 81.7%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [ ]:
Will use a new architecture to classify the images. First Conv Layer will take 28x28 input and convert ot 4x4 output

In [57]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
beta = 1e-3
graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  

  #helper functions to intialize weights, biases, convlution layer and max_pooling layer
  def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

  def bias_variable(val, shape):
    return tf.Variable(tf.constant(val, shape=shape))
                                   
  def conv2d(x , W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
  
  def max_pool(conv_layer):
    return tf.nn.max_pool(conv_layer, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

                                   
  # Variables.
  weights_conv_1 = weight_variable([patch_size, patch_size, num_channels, depth])
  bias_conv1 = bias_variable(0.0,[depth])
  weights_conv_2 = weight_variable([patch_size, patch_size, depth, depth])
  biases_conv2 = bias_variable(1.0,[depth])
  weights_hidden_1 = weight_variable([image_size // 4 * image_size // 4 * depth, num_hidden])
  biases_hidden_1 = bias_variable(1.0,[num_hidden])
  weights_layer_final = weight_variable([num_hidden, num_labels])
  biases_final = bias_variable(1.0,[num_labels])
  global_step = tf.Variable(0)

    
  
        
  def model(data, keep_prob):
    #input 28x28
    conv_1 = conv2d(data, weights_conv_1)
    hidden_conv_1 = tf.nn.relu(conv_1 + bias_conv1)
    #input 28x28 image since we are using same padding
    max_pool_1 = max_pool(hidden_conv_1)
    # input is 14x14 image
    conv_2 = conv2d(max_pool_1, weights_conv_2)
    hidden_conv_2 = tf.nn.relu(conv_2 + biases_conv2)
    #input is 14x14
    max_pool_2 = max_pool(hidden_conv_2)
    #input is 7x7 after another layer od pooilin
    shape = max_pool_2.get_shape().as_list()
    reshape = tf.reshape(max_pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden_3 = tf.nn.relu(tf.matmul(reshape, weights_hidden_1) + biases_hidden_1)
    drop_layer = tf.nn.dropout(hidden_3, keep_prob)
    return tf.matmul(drop_layer, weights_layer_final) + biases_final
  
  # Training computation.
  logits = model(tf_train_dataset, 1.0)

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta * (tf.nn.l2_loss(weights_conv_1) + 
                                                                                          tf.nn.l2_loss(weights_conv_2) + \
              tf.nn.l2_loss(weights_hidden_1) +tf.nn.l2_loss(weights_layer_final))
    
  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.05, global_step, 4000, 0.85, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset,1.0))
  test_prediction = tf.nn.softmax(model(tf_test_dataset,1.0))

In [59]:
num_steps = 49001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 4000 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.804635
Minibatch accuracy: 18.8%
Validation accuracy: 13.8%
Minibatch loss at step 4000: 0.465917
Minibatch accuracy: 81.2%
Validation accuracy: 87.5%
Minibatch loss at step 8000: 0.732219
Minibatch accuracy: 81.2%
Validation accuracy: 88.3%
Minibatch loss at step 12000: 0.784028
Minibatch accuracy: 81.2%
Validation accuracy: 88.9%
Minibatch loss at step 16000: 0.756824
Minibatch accuracy: 81.2%
Validation accuracy: 89.7%
Minibatch loss at step 20000: 0.317924
Minibatch accuracy: 87.5%
Validation accuracy: 89.7%
Minibatch loss at step 24000: 0.409307
Minibatch accuracy: 93.8%
Validation accuracy: 89.9%
Minibatch loss at step 28000: 0.608554
Minibatch accuracy: 75.0%
Validation accuracy: 90.2%
Minibatch loss at step 32000: 0.613297
Minibatch accuracy: 87.5%
Validation accuracy: 90.4%
Minibatch loss at step 36000: 0.355621
Minibatch accuracy: 81.2%
Validation accuracy: 90.8%
Minibatch loss at step 40000: 0.292018
Minibatch accuracy: 93.8%
Validatio

96.4 is my best result on this dataset. I think we can improve this with a little more parameter tunig using the validation set, but this is good for now.